# API (With Key) Basics

- Some APIs use a key
- Keep the Key secret
- Limits on requests
- Cache results


## 1. Learning Goals
By the end of this section, you will be able to:
- Call a public API that **requires an API key**.
- Pass the API key securely in a request (header or query parameter).
- Handle environment variables for sensitive credentials.
- Convert JSON responses into a table (e.g., DataFrame).


---

## 2. Request → Response Mental Model
The mental model is the same, but now your **credentials** are part of the conversation.

- **Endpoint**: the exact URL you call.
- **Method**: often `GET` for retrieving data.
- **Headers**: key–value pairs in the request header (e.g., API key).
- **Query parameters**: filters and options added after `?` in the URL.
- **Status code**: tells you if the request worked (`200 OK`) or failed (`4xx/5xx`).
- **Body**: the data returned, usually JSON.

Example request URL:

This tutorial uses the [NASA API](https://api.nasa.gov/)

## 3. Vocabulary for APIs
## Vocabulary for API with Key (NASA NEO Example)

| Term                      | Why it matters                                                                 |
|---------------------------|--------------------------------------------------------------------------------|
| API Key                   | A secret token you include in your request to prove you have permission to use the API. |
| Endpoint                  | The specific URL you call (e.g., `https://api.nasa.gov/neo/rest/v1/neo/browse`). |
| Query Parameters          | Added api key to the endpoint URL, like `?api_key=YOUR_KEY&size=5`.      |
| JSON                      | The data format returned by the API (nested dictionaries/lists in Python).     |
| Near Earth Object (NEO)   | NASA’s term for an asteroid or comet that passes near Earth — the data we’re exploring. |
| Absolute Magnitude (H)    | A measure of an asteroid’s brightness, used to estimate its size.              |
| Estimated Diameter        | The object’s size, provided in kilometers, meters, miles, and feet.            |
| Potentially Hazardous     | Boolean flag (`True`/`False`) showing whether NASA considers the asteroid hazardous. |
| Close Approach Data       | Details about when and how close an asteroid will pass near Earth.             |
| Relative Velocity         | The speed of the asteroid at close approach, given in km/s, km/h, and mph.     |
| Miss Distance             | The distance between Earth and the asteroid during close approach (AU, lunar distances, km, miles). |
| DataFrame (pandas)        | A table-like structure in Python, useful for organizing API results for analysis. |

| %run | magic function %run <filename> , %lsmagic|


In [22]:
# import libraries
import pandas as pd
import requests
import pprint
#%run config.py

Define location from where you want weather data from.

Form URL from NASA API endpoint key

In [ ]:
# Example 1: request with key:
NASA_API_KEY = "Your_Key_HERE"

# Load API key from environment variable 
url = "https://api.nasa.gov/neo/rest/v1/neo/browse?api_key={NASA_API_KEY}"
response = requests.get(url)
data = response.json()

# Example 2: request data for a single day
params = {
    "start_date": "2025-01-01",
    "end_date": "2025-01-01",
    "api_key": NASA_API_KEY
}

response = requests.get(url, params=params)
data = response.json()

In [24]:
print(data.keys())

dict_keys(['links', 'page', 'near_earth_objects'])


In [25]:
# Pretty print the first asteroid entry
pp = pprint.PrettyPrinter(indent=2, width=100)
pp.pprint(data["near_earth_objects"][0])

{ 'absolute_magnitude_h': 10.4,
  'close_approach_data': [ { 'close_approach_date': '1900-12-27',
                             'close_approach_date_full': '1900-Dec-27 01:30',
                             'epoch_date_close_approach': -2177879400000,
                             'miss_distance': { 'astronomical': '0.3149291693',
                                                'kilometers': '47112732.928149391',
                                                'lunar': '122.5074468577',
                                                'miles': '29274494.7651919558'},
                             'orbiting_body': 'Earth',
                             'relative_velocity': { 'kilometers_per_hour': '20083.0290749201',
                                                    'kilometers_per_second': '5.5786191875',
                                                    'miles_per_hour': '12478.8132604691'}},
                           { 'close_approach_date': '1907-11-05',
                             

In [27]:
# Turn selected NEO fields into a pandas DataFrame

neos = data["near_earth_objects"]

records = []
for obj in neos:
    records.append({
        "id": obj["id"],
        "name": obj["name"],
        "magnitude": obj["absolute_magnitude_h"],
        "diameter_min_km": obj["estimated_diameter"]["kilometers"]["estimated_diameter_min"],
        "diameter_max_km": obj["estimated_diameter"]["kilometers"]["estimated_diameter_max"],
        "hazardous": obj["is_potentially_hazardous_asteroid"]
    })

df = pd.DataFrame(records)
print(df.head())


        id                    name  magnitude  diameter_min_km  \
0  2000433      433 Eros (A898 PA)      10.40        22.108281   
1  2000719    719 Albert (A911 TB)      15.59         2.025606   
2  2000887    887 Alinda (A918 AA)      13.81         4.597852   
3  2001036  1036 Ganymed (A924 UB)       9.18        38.775283   
4  2001221    1221 Amor (1932 EA1)      17.37         0.892391   

   diameter_max_km  hazardous  
0        49.435619      False  
1         4.529393      False  
2        10.281109      False  
3        86.704169      False  
4         1.995446      False  
